In [1]:
from Data_Extraction_Visualization import XMLExtraction, Visualize
from Data_Retriever_Seg import DataRetriever
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from Resnet_UNet import ResNetUNet
import torch
import time
import torch.nn as nn
import os
import numpy as np
%matplotlib inline

In [2]:
CUDA_LAUNCH_BLOCKING="1"

In [3]:
my_path = '.\ANNOTATIONS'
XML_obj = XMLExtraction(my_path)
data = XML_obj.df

In [4]:
data.head()

,Name,image_path,annotation_path,label
0,crazing_1,.\IMAGES\crazing_1.jpg,.\ANNOTATIONS\crazing_1.xml,crazing
1,crazing_10,.\IMAGES\crazing_10.jpg,.\ANNOTATIONS\crazing_10.xml,crazing
2,crazing_100,.\IMAGES\crazing_100.jpg,.\ANNOTATIONS\crazing_100.xml,crazing
3,crazing_101,.\IMAGES\crazing_101.jpg,.\ANNOTATIONS\crazing_101.xml,crazing
4,crazing_102,.\IMAGES\crazing_102.jpg,.\ANNOTATIONS\crazing_102.xml,crazing


In [5]:
data_crazing = data[data['label'] == 'crazing']

In [6]:
train_crazing, test_crazing = train_test_split(data[data['label'] == 'crazing'],
                                              test_size=0.33,
                                              random_state=123)

In [7]:
tmp_path = train_crazing["image_path"].tolist()
tmp_target = train_crazing["annotation_path"].tolist()
train_data_retriever = DataRetriever(
    tmp_path,
    tmp_target
)
# dataloader training
dataloader_training = DataLoader(dataset=train_data_retriever,
                                 batch_size=10,
                                 shuffle=True)

In [8]:
tmp_path = test_crazing["image_path"].tolist()
tmp_target = test_crazing["annotation_path"].tolist()
test_data_retriever = DataRetriever(
    tmp_path,
    tmp_target
)
# dataloader test
dataloader_test = DataLoader(dataset=test_data_retriever,
                                 batch_size=10,
                                 shuffle=True)

In [9]:
Base_Model1 = ResNetUNet(n_class=1)

In [10]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

In [11]:
model = Base_Model1.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), weight_decay=1e-4, lr = 0.001, momentum=0.9)

In [12]:
%%time
for epoch in range(5):      
    model.train()         
    for ii, (data, target) in enumerate(dataloader_training):                         
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)  
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()          
    print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

RuntimeError: CUDA out of memory. Tried to allocate 800.00 MiB (GPU 0; 2.00 GiB total capacity; 945.84 MiB already allocated; 361.76 MiB free; 10.16 MiB cached)